In [8]:
import pandas as pd
import numpy as np
import os
from keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from keras.models import Model
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.segmentation import mark_boundaries
import lime
from lime import lime_image
import cv2

In [9]:
data_path = 'Data'

In [10]:
baseModel = InceptionResNetV2(input_shape=(224,224,3), weights='imagenet', include_top=False)

for layer in baseModel.layers:
  layer.trainable = False

headModel = baseModel.output
headModel = AveragePooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(3, activation='softmax')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [ ]:
print("Base Model loaded !")
baseModel.summary()

In [ ]:
print("Total Model loaded !")
model.summary()

In [ ]:
from keras.optimizers import Adam
Optimizer=Adam(learning_rate=0.001)
model.compile(optimizer=Optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                    width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   rotation_range=15,
                                   horizontal_flip = True,
                                   vertical_flip=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.1,
                                   fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale = 1./255)

training_gen = train_datagen.flow_from_directory(data_path + '/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

test_gen = test_datagen.flow_from_directory(data_path + '/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            shuffle = False)

In [ ]:
history1 = model.fit(training_gen,validation_data=test_gen,epochs=50)

In [ ]:
model.evaluate(training_gen,verbose=1)

In [ ]:
model.evaluate(test_gen,verbose=1)

In [ ]:
get_acc = history1.history['accuracy']
value_acc = history1.history['val_accuracy']
get_loss = history1.history['loss']
validation_loss = history1.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
model.save("covidModelMAIN1STVgg16SAIF.h5")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title("model accuracy of VGG16")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy"])
plt.show()
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title("model loss of VGG16")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss","Validation Loss"])
plt.show()

In [ ]:
from random import shuffle
from tqdm import tqdm  
import scipy
import skimage
from skimage.transform import resize

In [ ]:
from sklearn.metrics import precision_score, recall_score,\
                            confusion_matrix, classification_report, \
                            accuracy_score, f1_score

In [ ]:
class_dict = {0:'COVID19',1:'NORMAL',2:'PNEUMONIA'}

In [ ]:
pred=model.predict(test_gen)

In [ ]:
print(len(pred))

In [ ]:
y_test=[]
for i in range(26):
    y_test.extend(test_gen.__getitem__(i)[1])

In [ ]:
print(len(y_test))
y_test=np.array(y_test)

In [ ]:
y_test=np.argmax(y_test,axis=1)
pred= np.argmax(pred,axis=1)

In [ ]:
print("pred \n",len(pred))
print("y_test \n",len(y_test))

In [ ]:
print("y_test \n",y_test)
print("pred \n",pred)

In [ ]:
print(classification_report(test_gen.classes,pred))

In [ ]:
print(confusion_matrix(test_gen.classes,pred))

In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix(test_gen.classes,pred), annot=True)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
load_img("Data/test/COVID19/COVID19(461).jpg",target_size=(224,224))

In [ ]:
image=load_img("Data/test/COVID19/COVID19(461).jpg",target_size=(224,224))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [ ]:
reverse_mapping={0:'COVID19', 1:'NORMAL', 2:'PNEUMONIA'}

def mapper(value):
    return reverse_mapping[value]

prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
print("Prediction is {}.".format(move_name))

# actual

In [ ]:
prediction4 = np.argmax(model.predict(test_gen), axis = -1)
prediction4

In [ ]:
y_test4=[]
for i in range(41):
    y_test4.extend(test_gen.__getitem__(i)[1])
y_test4=np.array(y_test4)
plt.figure(figsize=(20,10))
fig = plt.gcf()
ncolumns,nrows=5,5
fig.set_size_inches(ncolumns*5, nrows*5)
for i in range(25):
    plt.subplot(nrows, ncolumns, i + 1)
    plt.imshow(test_gen.__getitem__(0)[0][i],cmap='gray')
    plt.title(f"REAL: {getLabel(y_test4[i])   } vs  PREDICT: {getLabel(prediction4[i])}")

In [ ]:
from random import shuffle
from tqdm import tqdm  
import scipy
import skimage
from skimage.transform import resize

In [ ]:
plt.figure(figsize=(20,10))
for i in range(0,9):
    
    plt.subplot(3, 3, i + 1)
    
    plt.imshow(test_gen.__getitem__(0)[0][i],cmap='gray')
    plt.title(f"   Real: {getLabel(y_test[i])   } Vs  Predict: {getLabel(pred[i])}")

In [ ]:
import seaborn as sns
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve, auc